# DistilBERT SQuAD 2.0 Model

In [ ]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

squad = load_dataset("squad_v2", split="train[:20000]")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


In [ ]:
squad = squad.train_test_split(test_size=0.2)

In [ ]:
squad["train"][0]

{'id': '5a8dc135df8bba001a0f9bc9',
 'title': 'The_Legend_of_Zelda:_Twilight_Princess',
 'context': 'On release, Twilight Princess was considered to be the greatest Zelda game ever made by many critics including writers for 1UP.com, Computer and Video Games, Electronic Gaming Monthly, Game Informer, GamesRadar, IGN and The Washington Post. Game Informer called it "so creative that it rivals the best that Hollywood has to offer". GamesRadar praised Twilight Princess as "a game that deserves nothing but the absolute highest recommendation". Cubed3 hailed Twilight Princess as "the single greatest videogame experience". Twilight Princess\'s graphics were praised for the art style and animation, although the game was designed for the GameCube, which is technically lacking compared to the next generation consoles. Both IGN and GameSpy pointed out the existence of blurry textures and low-resolution characters. Despite these complaints, Computer and Video Games felt the game\'s atmosphere was s

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# Define a function to extract the inputs and labels for fine-tuning bert from squad dataset
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]  # Extract the questions from the squad dataset
    inputs = tokenizer(
        questions,
        examples["context"],  # Extract the context from the squad dataset
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")  # each tuple contains two integers that represent the start and end character positions of the corresponding token in the original input text.
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        if answer["answer_start"] != []:
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])
            sequence_ids = inputs.sequence_ids(i)

            # Find the start and end of the context
            idx = 0
            while sequence_ids[idx] != 1:
                idx += 1
            context_start = idx
            while sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1

            # If the answer is not fully inside the context, label it (0, 0)
            if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Otherwise it's the start and end token positions
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)
        else:
            start_positions.append(0)
            end_positions.append(0)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
# Calling the preprocess_function on squad dataset
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [ ]:
# Collate the examples into a batch using data collator
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator()

In [ ]:
# Load the pre-trained bert model
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

In [ ]:
import time

start_time = time.time()

# Perform the training and set up the parameters
training_args = TrainingArguments(
    output_dir="distilbert_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

end_time = time.time()

Cloning https://huggingface.co/dennischan/distilbert_model into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/253M [00:00<?, ?B/s]

Download file runs/Apr18_01-36-27_bea688678b56/1681781799.2555928/events.out.tfevents.1681781799.bea688678b56.…

Download file runs/Apr18_02-24-52_96ae66fd892e/events.out.tfevents.1681784758.96ae66fd892e.1043.0: 100%|######…

Download file runs/Apr18_02-24-52_96ae66fd892e/1681784758.836044/events.out.tfevents.1681784758.96ae66fd892e.1…

...

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Clean file runs/Apr18_01-36-27_bea688678b56/events.out.tfevents.1681781799.bea688678b56.1839.0:  19%|#9       …

Clean file pytorch_model.bin:   0%|          | 1.00k/253M [00:00<?, ?B/s]

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.708900,1.507103
2,1.229400,1.312297
3,0.971500,1.315607


In [ ]:
print('The execution time for DistilBERT with squad_2.0 dataset is: ', round((end_time - start_time)/60, 2),'mins')

The execution time for DistilBERT with squad_2.0 dataset is:  36.95 mins


In [ ]:
question1 = "What are the symptoms of infecting covid?"
context1 = "Cough (68%), fever/chills (58%), and shortness of breath (37%) reported most often. Median number of days from symptom onset to laboratory confirmed COVID-19 diagnosis was 4 days (range 0-26 days)"

In [ ]:
from transformers import pipeline
question_answerer = pipeline("question-answering", model="distilbert_model")

answer1 = question_answerer(question=question1, context=context1)['answer']
question_answerer(question=question1, context=context1)

{'score': 0.17433443665504456, 'start': 0, 'end': 5, 'answer': 'Cough'}

In [ ]:
question2 = "What causes death from Covid-19?"
context2 = "Since the end of 2019 the Coronavirus Disease 2019 (COVID-19), caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2), has spread globally affecting people worldwide. Patients with severe CO VID-19 require intensive care unit (ICU) admission for acute respiratory failure and over 10% need noninvasive and invasive mechanical ventilation."


In [ ]:
answer2 = question_answerer(question=question2, context=context2)['answer']
question_answerer(question=question2, context=context2)

{'score': 0.12134205549955368,
 'start': 73,
 'end': 106,
 'answer': 'severe acute respiratory syndrome'}

In [ ]:
question3 = "Where is the origin of covid?"
context3 = "On December 31, 2019, the first case of what would later be known as SARS-CoV-2 was detected in the city of Wuhan, China. By January 10, 2022, more than 305 million people had been infected with COVID- 19, leading to more than 5.4 million deaths."


In [ ]:
answer3 = question_answerer(question=question3, context=context3)['answer']
question_answerer(question=question3, context=context3)

{'score': 0.37993186712265015, 'start': 108, 'end': 113, 'answer': 'Wuhan'}

In [ ]:
question4 = "What is the origin of covid?"
context4 = "The coronavirus disease 2019 (COVID-19) is caused by a novel strain of SARS-CoV-2. More than 305 million people had been infected with the virus by January 10, 2022, leading to 5.4 million deaths."


In [ ]:
answer4 = question_answerer(question=question4, context=context4)['answer']
question_answerer(question=question4, context=context4)

{'score': 0.1552591174840927,
 'start': 4,
 'end': 28,
 'answer': 'coronavirus disease 2019'}

In [ ]:
question5 = "When was covid first discovered?"
context5 = "In December 2019, a new coronavirus disease named COVID-19 by the World Health Organization broke out in Wuhan, China. There is currently no evidence to support that these drugs may be effective in discouraging Covid-19."

In [ ]:
answer5 = question_answerer(question=question5, context=context5)['answer']
question_answerer(question=question5, context=context5)

{'score': 0.709756076335907, 'start': 3, 'end': 16, 'answer': 'December 2019'}

# Reference

Hugging Face. Question answering — transformers doc-
umentation. https : / / huggingface . co / docs /
transformers / tasks / question _ answering,
n.d. Accessed: Apr. 18, 2023.